In [ ]:
import glob
import json
import os
import pandas as pd

def get_method_df(method: str, n: int) -> pd.DataFrame:

    method_df = pd.DataFrame()
    for fp in sorted(glob.glob(os.path.join("../results/", method, f"n={n}/", "l=*.json"))):
        with open(fp, "r") as f:
            result_json = json.load(f)
        if len(result_json["results"]) == 0:
            continue
        result_df = pd.DataFrame(result_json["results"])
        result_df["l"] = result_json["l"]
        method_df = pd.concat([method_df, result_df])
    
    return method_df

In [ ]:
n = 50

random_df = get_method_df(method="random", n=n)
two_df = get_method_df(method="two", n=n)
modal_df = get_method_df(method="modal", n=n)

In [ ]:
stats = {
    "runtime": {
        "random": [random_df[random_df.l == l].runtime.median() for l in random_df.l.unique()],
        "two": [two_df[two_df.l == l].runtime.median() for l in two_df.l.unique()],
        "modal": [modal_df[modal_df.l == l].runtime.median() for l in modal_df.l.unique()]
    },
    "calls": {
         "random": [random_df[random_df.l == l].n_calls.median() for l in random_df.l.unique()],
        "two": [two_df[two_df.l == l].n_calls.median() for l in two_df.l.unique()],
        "modal": [modal_df[modal_df.l == l].n_calls.median() for l in modal_df.l.unique()]
    }
}

In [ ]:
import plotly.graph_objects as go
from typing import List

def compare_performance(method_1: str, method_2: str, metric: str, x: List[int], y_unit: str = "", title: str = None):

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=x, y=stats[metric][method_1], name=method_1))
    fig.add_trace(go.Scatter(x=x, y=stats[metric][method_2], name=method_2))

    fig.update_layout(
        title=title,
        xaxis_title="Ratio",
        yaxis_title=metric.capitalize() + f" ({y_unit})",
        legend_title="Heuristic"
    )

    return fig

In [ ]:
fig = compare_performance("random", "modal", metric="runtime", x=random_df.l.unique() / n, y_unit="s", title=f"Runtime (n={n})")
fig.write_image("../plots/runtime_random_v_modal.png")
fig.show()

In [ ]:
fig = compare_performance("two", "modal", metric="runtime", x=two_df.l.unique() / n, y_unit="s", title=f"Runtime (n={n})")
fig.write_image("../plots/runtime_two_v_modal.png")
fig.show()

In [ ]:
fig = compare_performance("random", "modal", metric="calls", x=random_df.l.unique() / n, y_unit="#", title=f"Splitting calls (n={n})")
fig.write_image("../plots/calls_random_v_modal.png")
fig.show()

In [ ]:
fig = compare_performance("two", "modal", metric="calls", x=two_df.l.unique() / n, y_unit="#", title=f"Splitting calls (n={n})")
fig.write_image("../plots/calls_two_v_modal.png")
fig.show()